<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/idash_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,235 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,104 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,677 kB in 4s (704 kB/s)
Reading package lists... Done


In [31]:
## 무조건 해야하는 부분 2
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
## 한개씩 클릭해서 크롤링 하려함 (구조가 통일된 현행법만 크롤링하기 위해)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

driver.implicitly_wait(10)
driver.get("https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01")

laws_per_cat = {}
# for i in range(2, 47):
for i in range(2, 47):
  category = driver.find_element(By.CSS_SELECTOR, f"#tree_{i}_span") # 카테고리 선택
  cat = category.text
  laws_per_cat[cat.split()[1]] = int(cat.split('(')[-1].replace(')',''))

# category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
table_list = [f'#\\{i} > td:nth-child(2) > p > a' for i in range(31,40)] + [f'#\\31 {i} > td:nth-child(2) > p > a' for i in range(10)] + ['#\\32 0 > td:nth-child(2) > p > a']
pages = [f'#container > div > div.paging > a:nth-child({i})' for i in range(4,14)]

cur_sum_pages = 1
total_law = 1

while total_law <= 2341:
  for c in category_list:
    cur_law = 1
    final_df = pd.DataFrame(columns=['cat','name','eng','kor'])

    driver.switch_to.default_content()
    category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, c))) # 카테고리 선택
    # category = driver.find_element(By.CSS_SELECTOR, c)
    print(category.text)
    cat = category.text.split()[1] # 실제 카테고리 ex) 헌법
    num = laws_per_cat[cat]
    driver.execute_script("arguments[0].click();", category)

    driver.switch_to.frame('indexFrame') # html안에 html로 스위치
    mode = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#selectLangKor"))) # 이건 한국어, 영어 법령으로 보겠다는 의미
    driver.execute_script("arguments[0].click();", mode)

    for p in pages:
      for ind in table_list:
        law_name = re.sub(r"[^ㄱ-ㅣ가-힣\s]", "",  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ind.split(' > p')[0]))).text).strip()
        driver.find_element(By.CSS_SELECTOR, ind).send_keys(Keys.ENTER) # 여기가 이제 1번부터 클릭해서 새로운탭으로 들어가는 코드

        driver.switch_to.window(driver.window_handles[-1]) # 새로 열린 탭으로 작업환경 전환
        driver.switch_to.frame('lawViewContent') # html안에 html로 스위치, 즉 법령내용이 포함된 html로 접근 가능

        # req = driver.page_source
        # soup = BeautifulSoup(req, 'html.parser')
        # corpus = soup.find_all(['span', 'a']) # 내용만 긁자

        hang = driver.find_elements(By.CLASS_NAME, "hang")
        # ho = driver.find_elements(By.CLASS_NAME, "ho")
        none = driver.find_elements(By.CLASS_NAME, "none")
        corpus = hang + none

        # eng = ''
        # kor = ''
        eng = []
        kor = []
        eng_num = 0
        kor_num = 0
        for i in corpus: # corpus에서 tag 하나씩 eng, kor 매칭해서 저장하기
          clean_text = i.text
          if clean_text.startswith('['):
            continue
          if len(re.sub('[^ㄱ-ㅣ가-힣]','', clean_text)) == 0:
            if len(eng) != 0 and len(kor) != 0 and (eng_num == kor_num):
              # final_df = pd.concat([final_df, pd.DataFrame.from_dict([{'cat': cat, 'name': law_name, 'eng': eng.strip(), 'kor': kor.strip()}])])
              final_df = pd.concat([final_df, pd.DataFrame(zip([cat]*len(eng), [law_name]*len(eng), eng, kor), columns=final_df.columns)])
              eng = []
              kor = []
              eng_num = 0
              kor_num = 0
            elif len(eng) != 0 and len(kor) != 0 and (eng_num != kor_num):
              eng = []
              kor = []
              eng_num = 0
              kor_num = 0
            # eng += clean_text
            eng.append(clean_text)
            eng_num += 1
          else:
            if eng_num == 0:
              kor = []
              kor_num = 0
              continue
            # kor += clean_text
            kor.append(clean_text)
            kor_num += 1

        print(f'a number of crawled laws : {cur_law}')
        cur_law += 1
        total_law += 1
        driver.close() # 현재 탭 닫기
        driver.switch_to.window(driver.window_handles[0]) # 원래 탭으로 전환, 즉 법령 list가 있는 탭
        driver.switch_to.frame('indexFrame') # html안에 html로 스위치
        if cur_law > num:
          break
      if cur_law > num:
        break
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, p))).click() # 테이블에 있는 모든 법령을 크롤링 한 뒤 다음 페이지로 이동
      # driver.find_element(By.CSS_SELECTOR, p).click()
      cur_sum_pages += 1
      print(f'a number of crawled pages : {cur_sum_pages}')

    final_df = final_df.reset_index(drop=True)
    final_df.to_csv(f'/content/drive/MyDrive/KB/{cat}.csv')

제1편 헌법 (31)
a number of crawled laws : 1
a number of crawled laws : 2
a number of crawled laws : 3
a number of crawled laws : 4
a number of crawled laws : 5
a number of crawled laws : 6
a number of crawled laws : 7
a number of crawled laws : 8
a number of crawled laws : 9
a number of crawled laws : 10
a number of crawled laws : 11
a number of crawled laws : 12
a number of crawled laws : 13
a number of crawled laws : 14
a number of crawled laws : 15
a number of crawled laws : 16
a number of crawled laws : 17
a number of crawled laws : 18
a number of crawled laws : 19
a number of crawled laws : 20
a number of crawled pages : 2
a number of crawled laws : 21
a number of crawled laws : 22
a number of crawled laws : 23
a number of crawled laws : 24
a number of crawled laws : 25
a number of crawled laws : 26
a number of crawled laws : 27
a number of crawled laws : 28
a number of crawled laws : 29
a number of crawled laws : 30
a number of crawled laws : 31
제2편 국회 (10)
a number of crawled laws 